---

<p align="center">
  <img src="https://raw.githubusercontent.com/devicons/devicon/master/icons/python/python-original.svg" width="80"/>
</p>

<h1 align="center">Programação Orientada a Objetos</h1>

<h3 align="center">PhD. Julles Mitoura</h3>

<p align="center">
  <img src="https://img.shields.io/badge/Python-3776AB?style=for-the-badge&logo=python&logoColor=white"/>
  <img src="https://img.shields.io/badge/Jupyter-F37626?style=for-the-badge&logo=jupyter&logoColor=white"/>
  <img src="https://img.shields.io/badge/POO-4A90E2?style=for-the-badge"/>
</p>

---

## Aula 08 — Projeto Integrador (POO)

### Tema do projeto
**Sistema de Biblioteca (Catálogo, Usuários e Empréstimos)**

### Entidade de dados modelada (principal)
- **Item de Biblioteca** (ex.: `Livro`, `Revista`, `MidiaDigital`) com dados como `id`, `título`, `ano`, etc.

### Objetivo didático
Construir um mini-sistema que use, no mesmo projeto, todos os tópicos vistos:
- **Atributos, métodos e construtores**
- **Atributos e métodos de classe**
- **Encapsulamento** (validações, propriedades, invariantes)
- **Associação, agregação e composição**
- **Herança e reutilização**
- **Abstração e polimorfismo**

### Como trabalhar
O projeto está particionado em partes. Em cada parte:
- você implementa um conjunto pequeno de classes/métodos
- roda os exemplos
- e marca o checklist de “entregas”


## Partes do projeto

- **Parte 01**: Desenvolvendo a classe básica (`Livro`) — atributos, construtor, métodos simples
- **Parte 02**: Atributos e métodos de classe — IDs, contadores, fábricas (`@classmethod`)
- **Parte 03**: Encapsulamento — atributos privados, `@property`, validação, exceções
- **Parte 04**: Associação e agregação — `Autor` ↔ `Livro` (lista de autores no livro)
- **Parte 05**: Composição — `Emprestimo` (criado e gerenciado pelo sistema)
- **Parte 06**: Herança — `ItemBiblioteca` → `Livro`, `Revista`, `MidiaDigital`
- **Parte 07**: Abstração e polimorfismo — `ItemEmprestavel(ABC)` e regras diferentes de multa/prazo
- **Parte 08**: Classe de orquestração (`Biblioteca`) — cadastro, empréstimo, devolução
- **Parte 09 (opcional)**: Persistência simples — salvar/carregar em JSON usando `to_dict()/from_dict()`

> Ao final, você terá um “núcleo” de domínio (entidades) e um “serviço” (`Biblioteca`) que coordena as operações.


## Parte 01 — Classe básica: `Livro`

### Requisitos
- Representar um livro com **atributos** (título, ano, páginas)
- Ter um **construtor** (`__init__`)
- Ter **métodos** simples (`descricao()`, `to_dict()`)

### Entregas
- [ ] Implementar a classe `Livro`
- [ ] Instanciar 2 livros e imprimir as descrições


In [ ]:
class Livro:
    def __init__(self, titulo: str, ano: int, paginas: int):
        self.titulo = titulo
        self.ano = ano
        self.paginas = paginas

    def descricao(self) -> str:
        return f"{self.titulo} ({self.ano}) — {self.paginas} págs."

    def to_dict(self) -> dict:
        return {
            "titulo": self.titulo,
            "ano": self.ano,
            "paginas": self.paginas,
        }


l1 = Livro("Dom Casmurro", 1899, 256)
l2 = Livro("Capitães da Areia", 1937, 272)

print(l1.descricao())
print(l2.descricao())
print(l1.to_dict())

## Parte 02 — Atributos e métodos de classe

### Ideia
Vamos dar identidade aos livros e registrar estatísticas da classe.

### Requisitos
- Cada `Livro` deve ter um `id` gerado automaticamente
- A classe deve manter um contador (`total_criados`)
- Criar um método de classe `criar()` como fábrica

### Entregas
- [ ] Adicionar `total_criados` (atributo de classe)
- [ ] Adicionar geração de `id` com um contador interno
- [ ] Implementar `@classmethod criar(...)`


In [ ]:
class Livro:
    _proximo_id = 1  # atributo de classe ("contador" interno)
    total_criados = 0

    def __init__(self, titulo: str, ano: int, paginas: int):
        self.id = Livro._proximo_id
        Livro._proximo_id += 1
        Livro.total_criados += 1

        self.titulo = titulo
        self.ano = ano
        self.paginas = paginas

    @classmethod
    def criar(cls, dados: dict) -> "Livro":
        return cls(dados["titulo"], dados["ano"], dados["paginas"])

    def descricao(self) -> str:
        return f"#{self.id} — {self.titulo} ({self.ano}) — {self.paginas} págs."


l1 = Livro("Dom Casmurro", 1899, 256)
l2 = Livro("Capitães da Areia", 1937, 272)
l3 = Livro.criar({"titulo": "Vidas Secas", "ano": 1938, "paginas": 176})

print(l1.descricao())
print(l2.descricao())
print(l3.descricao())
print("Total criados:", Livro.total_criados)

## Parte 03 — Encapsulamento (validação e invariantes)

### Ideia
Agora vamos proteger o estado interno do objeto.

### Requisitos
- `titulo` não pode ser vazio
- `ano` deve ser um inteiro plausível (ex.: >= 1450 e <= ano atual)
- `paginas` deve ser positivo
- Use **atributos privados** e **propriedades** (`@property`)
- Em caso de erro, lançar `ValueError`

### Entregas
- [ ] Implementar encapsulamento com `__titulo`, `__ano`, `__paginas`
- [ ] Implementar getters/setters com validação
- [ ] Testar com 1 caso válido e 2 casos inválidos


In [ ]:
from datetime import date


class Livro:
    _proximo_id = 1
    total_criados = 0

    def __init__(self, titulo: str, ano: int, paginas: int):
        self.id = Livro._proximo_id
        Livro._proximo_id += 1
        Livro.total_criados += 1

        # usa setters (validação) durante a construção
        self.titulo = titulo
        self.ano = ano
        self.paginas = paginas

    @property
    def titulo(self) -> str:
        return self.__titulo

    @titulo.setter
    def titulo(self, valor: str) -> None:
        if not isinstance(valor, str) or not valor.strip():
            raise ValueError("titulo não pode ser vazio")
        self.__titulo = valor.strip()

    @property
    def ano(self) -> int:
        return self.__ano

    @ano.setter
    def ano(self, valor: int) -> None:
        ano_atual = date.today().year
        if not isinstance(valor, int) or valor < 1450 or valor > ano_atual:
            raise ValueError(f"ano inválido: {valor}")
        self.__ano = valor

    @property
    def paginas(self) -> int:
        return self.__paginas

    @paginas.setter
    def paginas(self, valor: int) -> None:
        if not isinstance(valor, int) or valor <= 0:
            raise ValueError("paginas deve ser um inteiro positivo")
        self.__paginas = valor

    def descricao(self) -> str:
        return f"#{self.id} — {self.titulo} ({self.ano}) — {self.paginas} págs."


# válido
l_ok = Livro("O Alquimista", 1988, 208)
print(l_ok.descricao())

# inválidos (descomente para testar)
# Livro("", 2000, 100)
# Livro("Teste", 1200, 100)
# Livro("Teste", 2000, -5)

## Parte 04 — Associação e agregação: `Autor` ↔ `Livro`

### Conceitos
- **Associação**: objetos se relacionam (um `Livro` referencia `Autor`)
- **Agregação**: “tem-um”, mas com vida independente (um `Autor` existe mesmo sem o `Livro`)

### Requisitos
- Criar a classe `Autor` (nome, ano_nascimento)
- O `Livro` deve manter uma **lista de autores**
- Métodos: `adicionar_autor()`, `listar_autores()`

### Entregas
- [ ] Implementar `Autor`
- [ ] Atualizar `Livro` para suportar autores
- [ ] Criar um livro com 2 autores e listar


In [ ]:
from datetime import date


class Autor:
    _proximo_id = 1

    def __init__(self, nome: str, ano_nascimento: int):
        self.id = Autor._proximo_id
        Autor._proximo_id += 1

        self.nome = nome
        self.ano_nascimento = ano_nascimento

    def __str__(self) -> str:
        return f"{self.nome} (n. {self.ano_nascimento})"


class Livro:
    _proximo_id = 1

    def __init__(self, titulo: str, ano: int, paginas: int):
        self.id = Livro._proximo_id
        Livro._proximo_id += 1

        self.titulo = titulo
        self.ano = ano
        self.paginas = paginas

        # agregação: autores existem independentemente do livro
        self.autores: list[Autor] = []

    @property
    def titulo(self) -> str:
        return self.__titulo

    @titulo.setter
    def titulo(self, valor: str) -> None:
        if not isinstance(valor, str) or not valor.strip():
            raise ValueError("titulo não pode ser vazio")
        self.__titulo = valor.strip()

    @property
    def ano(self) -> int:
        return self.__ano

    @ano.setter
    def ano(self, valor: int) -> None:
        ano_atual = date.today().year
        if not isinstance(valor, int) or valor < 1450 or valor > ano_atual:
            raise ValueError(f"ano inválido: {valor}")
        self.__ano = valor

    @property
    def paginas(self) -> int:
        return self.__paginas

    @paginas.setter
    def paginas(self, valor: int) -> None:
        if not isinstance(valor, int) or valor <= 0:
            raise ValueError("paginas deve ser um inteiro positivo")
        self.__paginas = valor

    def adicionar_autor(self, autor: Autor) -> None:
        if not isinstance(autor, Autor):
            raise TypeError("autor deve ser uma instância de Autor")
        if any(a.id == autor.id for a in self.autores):
            return
        self.autores.append(autor)

    def listar_autores(self) -> list[str]:
        return [str(a) for a in self.autores]

    def descricao(self) -> str:
        autores = ", ".join(self.listar_autores()) or "(sem autores cadastrados)"
        return f"#{self.id} — {self.titulo} ({self.ano}) — {self.paginas} págs. — Autores: {autores}"


a1 = Autor("Machado de Assis", 1839)
a2 = Autor("José de Alencar", 1829)

livro = Livro("Clássicos Brasileiros (Coletânea)", 2020, 400)
livro.adicionar_autor(a1)
livro.adicionar_autor(a2)

print(livro.descricao())

## Parte 05 — Composição: `Emprestimo`

### Conceito
- **Composição**: o “todo” controla o ciclo de vida da “parte”.

Aqui, o objeto `Emprestimo` só faz sentido **dentro do sistema** (gerenciado pela `Biblioteca`).

### Requisitos
- Criar `Usuario` (nome)
- Criar `Emprestimo` com: item, usuário, datas (empréstimo, prevista, devolução)
- Método `finalizar()` para registrar devolução

### Entregas
- [ ] Implementar `Usuario` e `Emprestimo`
- [ ] Criar um empréstimo e finalizar


In [ ]:
from datetime import date, timedelta


class Usuario:
    _proximo_id = 1

    def __init__(self, nome: str):
        self.id = Usuario._proximo_id
        Usuario._proximo_id += 1
        self.nome = nome

    def __str__(self) -> str:
        return f"{self.nome} (id={self.id})"


class Emprestimo:
    def __init__(
        self,
        item_titulo: str,
        usuario: Usuario,
        data_emprestimo: date | None = None,
        prazo_dias: int = 7,
    ):
        self.item_titulo = item_titulo
        self.usuario = usuario
        self.data_emprestimo = data_emprestimo or date.today()
        self.data_prevista = self.data_emprestimo + timedelta(days=prazo_dias)
        self.data_devolucao: date | None = None

    @property
    def ativo(self) -> bool:
        return self.data_devolucao is None

    def finalizar(self, data_devolucao: date | None = None) -> None:
        self.data_devolucao = data_devolucao or date.today()

    def __str__(self) -> str:
        status = "ativo" if self.ativo else f"finalizado em {self.data_devolucao}"
        return (
            f"Emprestimo(item={self.item_titulo!r}, usuario={self.usuario.nome!r}, "
            f"prevista={self.data_prevista}, status={status})"
        )


u = Usuario("Ana")
e = Emprestimo("Dom Casmurro", u, prazo_dias=3)
print(e)

e.finalizar(e.data_prevista + timedelta(days=2))
print(e)

## Parte 06 — Herança e reutilização: `ItemBiblioteca`

### Ideia
Vamos criar uma superclasse com o que é comum a qualquer item de biblioteca, e especializar nos tipos.

### Requisitos
- Criar `ItemBiblioteca` com:
  - `id` automático
  - `titulo` e `ano` com encapsulamento/validação
  - estado de disponibilidade (`disponivel`)
- Criar subclasses:
  - `Livro`
  - `Revista`
  - `MidiaDigital`

### Entregas
- [ ] Implementar a classe base e as subclasses
- [ ] Criar 1 item de cada tipo e imprimir


In [ ]:
from datetime import date


class ItemBiblioteca:
    _proximo_id = 1

    def __init__(self, titulo: str, ano: int):
        self.id = ItemBiblioteca._proximo_id
        ItemBiblioteca._proximo_id += 1

        self.titulo = titulo
        self.ano = ano
        self.__emprestado = False

    @property
    def titulo(self) -> str:
        return self.__titulo

    @titulo.setter
    def titulo(self, valor: str) -> None:
        if not isinstance(valor, str) or not valor.strip():
            raise ValueError("titulo não pode ser vazio")
        self.__titulo = valor.strip()

    @property
    def ano(self) -> int:
        return self.__ano

    @ano.setter
    def ano(self, valor: int) -> None:
        ano_atual = date.today().year
        if not isinstance(valor, int) or valor < 1450 or valor > ano_atual:
            raise ValueError(f"ano inválido: {valor}")
        self.__ano = valor

    @property
    def disponivel(self) -> bool:
        return not self.__emprestado

    def _marcar_emprestado(self) -> None:
        self.__emprestado = True

    def _marcar_devolvido(self) -> None:
        self.__emprestado = False

    def __str__(self) -> str:
        status = "disponível" if self.disponivel else "emprestado"
        return f"{self.__class__.__name__}(id={self.id}, titulo={self.titulo!r}, ano={self.ano}, {status})"


class Autor:
    def __init__(self, nome: str):
        self.nome = nome.strip()

    def __str__(self) -> str:
        return self.nome


class Livro(ItemBiblioteca):
    def __init__(self, titulo: str, ano: int, paginas: int, autores: list[Autor] | None = None):
        super().__init__(titulo, ano)
        self.paginas = paginas
        self.autores = autores or []

    @property
    def paginas(self) -> int:
        return self.__paginas

    @paginas.setter
    def paginas(self, valor: int) -> None:
        if not isinstance(valor, int) or valor <= 0:
            raise ValueError("paginas deve ser um inteiro positivo")
        self.__paginas = valor

    def __str__(self) -> str:
        base = super().__str__()
        autores = ", ".join(str(a) for a in self.autores) or "(sem autores)"
        return f"{base} — {self.paginas} págs. — autores: {autores}"


class Revista(ItemBiblioteca):
    def __init__(self, titulo: str, ano: int, edicao: int):
        super().__init__(titulo, ano)
        self.edicao = edicao

    @property
    def edicao(self) -> int:
        return self.__edicao

    @edicao.setter
    def edicao(self, valor: int) -> None:
        if not isinstance(valor, int) or valor <= 0:
            raise ValueError("edicao deve ser um inteiro positivo")
        self.__edicao = valor

    def __str__(self) -> str:
        return f"{super().__str__()} — edição {self.edicao}"


class MidiaDigital(ItemBiblioteca):
    def __init__(self, titulo: str, ano: int, formato: str):
        super().__init__(titulo, ano)
        self.formato = formato

    @property
    def formato(self) -> str:
        return self.__formato

    @formato.setter
    def formato(self, valor: str) -> None:
        if not isinstance(valor, str) or not valor.strip():
            raise ValueError("formato não pode ser vazio")
        self.__formato = valor.strip().lower()

    def __str__(self) -> str:
        return f"{super().__str__()} — formato {self.formato}"


it1 = Livro("Dom Casmurro", 1899, 256, autores=[Autor("Machado de Assis")])
it2 = Revista("Ciência Hoje", 2024, 321)
it3 = MidiaDigital("Curso de Python", 2023, "mp4")

print(it1)
print(it2)
print(it3)

## Parte 07 — Abstração e polimorfismo (regras de empréstimo)

### Ideia
Itens diferentes têm **regras diferentes** (prazo/multa). Usuários diferentes também.

### Requisitos
- Criar uma classe abstrata `ItemEmprestavel(ABC)` com:
  - `prazo_maximo_dias` (abstrato)
  - `calcular_multa(dias_atraso)` (abstrato)
- Fazer `Livro`, `Revista`, `MidiaDigital` implementarem essas regras
- Criar `Usuario(ABC)` e subclasses `Aluno` e `Professor` com limites diferentes

### Entregas
- [ ] Implementar `ItemEmprestavel` e `Usuario` (abstrações)
- [ ] Demonstrar polimorfismo: calcular multa para 3 tipos de item


In [ ]:
from abc import ABC, abstractmethod


class ItemEmprestavel(ItemBiblioteca, ABC):
    @property
    @abstractmethod
    def prazo_maximo_dias(self) -> int:
        raise NotImplementedError

    @abstractmethod
    def calcular_multa(self, dias_atraso: int) -> float:
        raise NotImplementedError


class Livro(ItemEmprestavel):
    def __init__(self, titulo: str, ano: int, paginas: int, autores: list[Autor] | None = None):
        super().__init__(titulo, ano)
        self.paginas = paginas
        self.autores = autores or []

    @property
    def paginas(self) -> int:
        return self.__paginas

    @paginas.setter
    def paginas(self, valor: int) -> None:
        if not isinstance(valor, int) or valor <= 0:
            raise ValueError("paginas deve ser um inteiro positivo")
        self.__paginas = valor

    @property
    def prazo_maximo_dias(self) -> int:
        return 7

    def calcular_multa(self, dias_atraso: int) -> float:
        dias = max(0, int(dias_atraso))
        return 1.50 * dias


class Revista(ItemEmprestavel):
    def __init__(self, titulo: str, ano: int, edicao: int):
        super().__init__(titulo, ano)
        self.edicao = edicao

    @property
    def edicao(self) -> int:
        return self.__edicao

    @edicao.setter
    def edicao(self, valor: int) -> None:
        if not isinstance(valor, int) or valor <= 0:
            raise ValueError("edicao deve ser um inteiro positivo")
        self.__edicao = valor

    @property
    def prazo_maximo_dias(self) -> int:
        return 3

    def calcular_multa(self, dias_atraso: int) -> float:
        dias = max(0, int(dias_atraso))
        return 1.00 * dias


class MidiaDigital(ItemEmprestavel):
    def __init__(self, titulo: str, ano: int, formato: str):
        super().__init__(titulo, ano)
        self.formato = formato

    @property
    def formato(self) -> str:
        return self.__formato

    @formato.setter
    def formato(self, valor: str) -> None:
        if not isinstance(valor, str) or not valor.strip():
            raise ValueError("formato não pode ser vazio")
        self.__formato = valor.strip().lower()

    @property
    def prazo_maximo_dias(self) -> int:
        return 5

    def calcular_multa(self, dias_atraso: int) -> float:
        dias = max(0, int(dias_atraso))
        return 2.00 * dias


class Usuario(ABC):
    _proximo_id = 1

    def __init__(self, nome: str):
        self.id = Usuario._proximo_id
        Usuario._proximo_id += 1
        self.nome = nome.strip()

    @property
    @abstractmethod
    def limite_emprestimos(self) -> int:
        raise NotImplementedError

    @property
    @abstractmethod
    def fator_multa(self) -> float:
        raise NotImplementedError

    def __str__(self) -> str:
        return f"{self.__class__.__name__}(id={self.id}, nome={self.nome!r})"


class Aluno(Usuario):
    @property
    def limite_emprestimos(self) -> int:
        return 2

    @property
    def fator_multa(self) -> float:
        return 1.0


class Professor(Usuario):
    @property
    def limite_emprestimos(self) -> int:
        return 5

    @property
    def fator_multa(self) -> float:
        return 0.7  # desconto na multa


itens: list[ItemEmprestavel] = [
    Livro("Dom Casmurro", 1899, 256, autores=[Autor("Machado de Assis")]),
    Revista("Ciência Hoje", 2024, 321),
    MidiaDigital("Curso de Python", 2023, "mp4"),
]

for item in itens:
    print(item.__class__.__name__, "prazo:", item.prazo_maximo_dias, "multa(4 dias):", item.calcular_multa(4))

u_aluno = Aluno("Ana")
u_prof = Professor("Bruno")
print(u_aluno, "limite:", u_aluno.limite_emprestimos)
print(u_prof, "limite:", u_prof.limite_emprestimos)

## Parte 08 — Orquestração: `Biblioteca` (cadastro, empréstimo e devolução)

### Ideia
A classe `Biblioteca` atua como “serviço” do sistema, coordenando as entidades:
- agrega `ItemEmprestavel`
- agrega `Usuario`
- **compõe** `Emprestimo` (ela cria/gerencia)

### Requisitos
- `cadastrar_item(item)`
- `cadastrar_usuario(usuario)`
- `emprestar(item_id, usuario_id, data_emprestimo=...)`
- `devolver(emprestimo_id, data_devolucao=...)` retornando a multa (se houver)

### Entregas
- [ ] Implementar `Biblioteca` e `Emprestimo` (versão final)
- [ ] Simular um empréstimo com atraso e imprimir a multa


In [ ]:
from datetime import date, timedelta


class RegraBibliotecaError(Exception):
    pass


class Emprestimo:
    _proximo_id = 1

    def __init__(self, item: ItemEmprestavel, usuario: Usuario, data_emprestimo: date | None = None):
        self.id = Emprestimo._proximo_id
        Emprestimo._proximo_id += 1

        self.item = item
        self.usuario = usuario
        self.data_emprestimo = data_emprestimo or date.today()
        self.data_prevista = self.data_emprestimo + timedelta(days=item.prazo_maximo_dias)
        self.data_devolucao: date | None = None

        # marca o item como emprestado (regra de consistência)
        self.item._marcar_emprestado()

    @property
    def ativo(self) -> bool:
        return self.data_devolucao is None

    def finalizar(self, data_devolucao: date | None = None) -> float:
        if not self.ativo:
            raise RegraBibliotecaError("empréstimo já finalizado")

        self.data_devolucao = data_devolucao or date.today()
        self.item._marcar_devolvido()

        atraso = self.dias_atraso
        multa_base = self.item.calcular_multa(atraso)
        return round(multa_base * self.usuario.fator_multa, 2)

    @property
    def dias_atraso(self) -> int:
        if self.data_devolucao is None:
            return 0
        return max(0, (self.data_devolucao - self.data_prevista).days)

    def __str__(self) -> str:
        status = "ativo" if self.ativo else f"finalizado em {self.data_devolucao}"
        return (
            f"Emprestimo(id={self.id}, item={self.item.titulo!r}, usuario={self.usuario.nome!r}, "
            f"prevista={self.data_prevista}, status={status})"
        )


class Biblioteca:
    def __init__(self, nome: str):
        self.nome = nome
        self.__itens: dict[int, ItemEmprestavel] = {}
        self.__usuarios: dict[int, Usuario] = {}
        self.__emprestimos: dict[int, Emprestimo] = {}

    def cadastrar_item(self, item: ItemEmprestavel) -> None:
        self.__itens[item.id] = item

    def cadastrar_usuario(self, usuario: Usuario) -> None:
        self.__usuarios[usuario.id] = usuario

    def _emprestimos_ativos_do_usuario(self, usuario_id: int) -> list[Emprestimo]:
        return [e for e in self.__emprestimos.values() if e.ativo and e.usuario.id == usuario_id]

    def emprestar(self, item_id: int, usuario_id: int, data_emprestimo: date | None = None) -> Emprestimo:
        item = self.__itens.get(item_id)
        if item is None:
            raise RegraBibliotecaError("item não encontrado")
        if not item.disponivel:
            raise RegraBibliotecaError("item indisponível")

        usuario = self.__usuarios.get(usuario_id)
        if usuario is None:
            raise RegraBibliotecaError("usuário não encontrado")

        ativos = self._emprestimos_ativos_do_usuario(usuario_id)
        if len(ativos) >= usuario.limite_emprestimos:
            raise RegraBibliotecaError("usuário atingiu o limite de empréstimos")

        emprestimo = Emprestimo(item, usuario, data_emprestimo=data_emprestimo)
        self.__emprestimos[emprestimo.id] = emprestimo
        return emprestimo

    def devolver(self, emprestimo_id: int, data_devolucao: date | None = None) -> float:
        emprestimo = self.__emprestimos.get(emprestimo_id)
        if emprestimo is None:
            raise RegraBibliotecaError("empréstimo não encontrado")
        return emprestimo.finalizar(data_devolucao=data_devolucao)

    def listar_itens(self) -> list[str]:
        return [str(i) for i in self.__itens.values()]

    def listar_usuarios(self) -> list[str]:
        return [str(u) for u in self.__usuarios.values()]

    def listar_emprestimos(self) -> list[str]:
        return [str(e) for e in self.__emprestimos.values()]


# --- Simulação ---
bib = Biblioteca("Biblioteca Central")

u1 = Aluno("Ana")
u2 = Professor("Bruno")

item1 = Livro("Dom Casmurro", 1899, 256, autores=[Autor("Machado de Assis")])
item2 = MidiaDigital("Curso de Python", 2023, "mp4")

bib.cadastrar_usuario(u1)
bib.cadastrar_usuario(u2)
bib.cadastrar_item(item1)
bib.cadastrar_item(item2)

emp = bib.emprestar(item1.id, u1.id, data_emprestimo=date(2026, 1, 1))
print(emp)

# devolvendo com atraso (prazo do Livro = 7 dias; devolvendo 4 dias depois do previsto)
multa = bib.devolver(emp.id, data_devolucao=date(2026, 1, 12))
print("Multa:", multa)
print("Item disponível depois da devolução?", item1.disponivel)

## Parte 09 (opcional) — Persistência simples (JSON)

### Ideia
Salvar e carregar o estado (catálogo + usuários) em um arquivo JSON.

### Requisitos
- Implementar `to_dict()` e `from_dict()`
- Salvar em `biblioteca.json`

### Entregas
- [ ] Serializar itens e usuários
- [ ] Salvar em arquivo e recarregar


In [ ]:
import json
from pathlib import Path


def item_to_dict(item: ItemEmprestavel) -> dict:
    base = {
        "tipo": item.__class__.__name__,
        "id": item.id,
        "titulo": item.titulo,
        "ano": item.ano,
    }

    if isinstance(item, Livro):
        base.update({
            "paginas": item.paginas,
            "autores": [a.nome for a in item.autores],
        })
    elif isinstance(item, Revista):
        base.update({"edicao": item.edicao})
    elif isinstance(item, MidiaDigital):
        base.update({"formato": item.formato})

    return base


def usuario_to_dict(usuario: Usuario) -> dict:
    return {
        "tipo": usuario.__class__.__name__,
        "id": usuario.id,
        "nome": usuario.nome,
    }


def item_from_dict(dados: dict) -> ItemEmprestavel:
    tipo = dados["tipo"]
    if tipo == "Livro":
        autores = [Autor(n) for n in dados.get("autores", [])]
        item = Livro(dados["titulo"], dados["ano"], dados["paginas"], autores=autores)
    elif tipo == "Revista":
        item = Revista(dados["titulo"], dados["ano"], dados["edicao"])
    elif tipo == "MidiaDigital":
        item = MidiaDigital(dados["titulo"], dados["ano"], dados["formato"])
    else:
        raise ValueError(f"tipo de item desconhecido: {tipo}")

    # mantém o id original (ajusta o contador global)
    item.id = int(dados["id"])
    ItemBiblioteca._proximo_id = max(ItemBiblioteca._proximo_id, item.id + 1)
    return item


def usuario_from_dict(dados: dict) -> Usuario:
    tipo = dados["tipo"]
    if tipo == "Aluno":
        u = Aluno(dados["nome"])
    elif tipo == "Professor":
        u = Professor(dados["nome"])
    else:
        raise ValueError(f"tipo de usuário desconhecido: {tipo}")

    u.id = int(dados["id"])
    Usuario._proximo_id = max(Usuario._proximo_id, u.id + 1)
    return u


def salvar_json(caminho: str | Path, itens: list[ItemEmprestavel], usuarios: list[Usuario]) -> None:
    payload = {
        "itens": [item_to_dict(i) for i in itens],
        "usuarios": [usuario_to_dict(u) for u in usuarios],
    }
    Path(caminho).write_text(json.dumps(payload, ensure_ascii=False, indent=2), encoding="utf-8")


def carregar_json(caminho: str | Path) -> tuple[list[ItemEmprestavel], list[Usuario]]:
    payload = json.loads(Path(caminho).read_text(encoding="utf-8"))
    itens = [item_from_dict(d) for d in payload.get("itens", [])]
    usuarios = [usuario_from_dict(d) for d in payload.get("usuarios", [])]
    return itens, usuarios


arquivo = Path("biblioteca.json")

itens = [
    Livro("Dom Casmurro", 1899, 256, autores=[Autor("Machado de Assis")]),
    Revista("Ciência Hoje", 2024, 321),
]
usuarios = [Aluno("Ana"), Professor("Bruno")]

salvar_json(arquivo, itens, usuarios)

itens2, usuarios2 = carregar_json(arquivo)
print("Recarregado — itens:")
for i in itens2:
    print("-", i)
print("Recarregado — usuários:")
for u in usuarios2:
    print("-", u)

## Checklist final (para entrega/avaliação)

- **Modelagem**
  - [ ] Diagrama mental: entidades (`ItemBiblioteca`, `Usuario`, `Emprestimo`) e relações
  - [ ] Pelo menos 3 tipos de itens e 2 tipos de usuários

- **POO (o que precisa aparecer no código)**
  - [ ] Construtores, atributos e métodos (Parte 01)
  - [ ] Atributos/métodos de classe (Parte 02)
  - [ ] Encapsulamento com `@property` + validação (Parte 03)
  - [ ] Associação/agregação (Parte 04)
  - [ ] Composição (Parte 05/08)
  - [ ] Herança (Parte 06)
  - [ ] Abstração + polimorfismo (Parte 07)

- **Extensões (opcionais)**
  - [ ] Relatório: itens emprestados, top usuários, histórico por item
  - [ ] Regras: reserva de item, bloqueio por multa acumulada
  - [ ] Persistência completa: salvar também empréstimos ativos
